In [1]:
import os

In [2]:
os.chdir('../..')

In [3]:
import dgl

Using backend: pytorch


In [4]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import DglNodePropPredDataset, Evaluator
import dgl

from dgl.nn.pytorch.conv import SAGEConv

In [5]:
dataset = DglNodePropPredDataset(name='ogbn-arxiv', 
                                 root='data/dataset_dgl/')
dataset

DglNodePropPredDataset(1)

In [6]:
torch.cuda.is_available()

True

In [7]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [8]:
graph = dataset[0][0]
graph

Graph(num_nodes=169343, num_edges=1166243,
      ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [9]:
split_idx = dataset.get_idx_split()

In [10]:
graph.add_edges(graph.edges()[1], graph.edges()[0])

In [11]:
graph

Graph(num_nodes=169343, num_edges=2332486,
      ndata_schemes={'year': Scheme(shape=(1,), dtype=torch.int64), 'feat': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={})

In [12]:
torch.cuda.set_device(device)

In [13]:
features = graph.ndata['feat'].cuda()
labels = dataset.labels.cuda()
train_mask = split_idx['train'].cuda()
val_mask = split_idx['valid'].cuda()
test_mask = split_idx['test'].cuda()

In [14]:
train_nid = train_mask.nonzero().squeeze()
val_nid = val_mask.nonzero().squeeze()
test_nid = test_mask.nonzero().squeeze()

<ipython-input-14-fa10932ea454>:1: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  train_nid = train_mask.nonzero().squeeze()


In [15]:
n_edges = graph.number_of_edges()

In [16]:
graph = graph.int().to(device)

In [17]:
n_iters = 3000
epochs = 1000
log_steps = 100
input_dim = features.shape[1]
hidden_channels = input_dim * 2
output_dim = dataset.num_classes
lr_rate = 0.001

In [18]:
class ThreeLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim, dropout):
        super(ThreeLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(SAGEConv(input_dim, hidden_channels, 'mean'))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, hidden_channels, 'mean'))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, output_dim, 'mean'))
        
        self.dropout = dropout
        

    def forward(self, graph, x):
        x = self.convs[0](graph, x)
        x = self.bns[0](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[1](graph, x)
        x = self.bns[1](x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.convs[2](graph, x)
        return x.log_softmax(dim=-1)

In [19]:
model = ThreeLayer(input_dim,
                   hidden_channels,
                   output_dim,
                   0.5).cuda()

In [20]:
def train(model, graph, features, train_mask, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(graph, features)[train_mask]
    loss = F.nll_loss(out, labels.squeeze(1)[train_mask])
    loss.backward()
    optimizer.step()

    return loss.item()

In [21]:
@torch.no_grad()
def test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator):
    model.eval()

    out = model(graph, features)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': labels[train_mask],
        'y_pred': y_pred[train_mask],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': labels[val_mask],
        'y_pred': y_pred[val_mask],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': labels[test_mask],
        'y_pred': y_pred[test_mask],
    })['acc']

    return train_acc, valid_acc, test_acc

In [22]:
evaluator = Evaluator(name='ogbn-arxiv')

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, graph, features, train_mask, optimizer)
    result = test(model, graph, features, labels, train_mask, val_mask, test_mask, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.3095, Train: 68.98%, Valid: 68.32% Test: 66.80%
Epoch: 200, Loss: 1.1143, Train: 71.45%, Valid: 70.05% Test: 68.31%
Epoch: 300, Loss: 1.0232, Train: 72.84%, Valid: 70.86% Test: 69.47%
Epoch: 400, Loss: 0.9613, Train: 73.96%, Valid: 71.44% Test: 70.06%
Epoch: 500, Loss: 0.9187, Train: 74.94%, Valid: 71.71% Test: 70.51%
Epoch: 600, Loss: 0.8829, Train: 75.81%, Valid: 71.92% Test: 70.62%
Epoch: 700, Loss: 0.8507, Train: 76.73%, Valid: 72.09% Test: 70.77%
Epoch: 800, Loss: 0.8213, Train: 77.57%, Valid: 72.31% Test: 71.13%
Epoch: 900, Loss: 0.7968, Train: 78.38%, Valid: 72.36% Test: 71.00%
Epoch: 1000, Loss: 0.7741, Train: 79.11%, Valid: 72.49% Test: 70.93%
